Fine-tune a GPT-2 model with LoRA adapters using Hugging Face Transformers, PEFT, and a small custom dataset


In [ ]:
pip install transformers datasets peft accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [ ]:
from pathlib import Path
print(Path.cwd())

/content


In [ ]:
import os
print(os.getcwd())

/content


In [ ]:
pip install --upgrade datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


In [ ]:
from datasets import Dataset, Features, Value

features = Features({'text': Value('string'), 'label': Value('int32')})
data = {'text': [], 'label': []}
dataset = Dataset.from_dict(data, features=features)  # ✅ No error

In [ ]:
!ls


data.json  sample_data


In [16]:
from datasets import Dataset
import json

with open('data.json', 'r') as f:
    data = json.load(f)

dataset = Dataset.from_list(data)
print(dataset[0])



{'input': 'Summarize the following legal clause.', 'output': 'This clause describes the conditions under which the contract is terminated.'}


In [17]:

def format_example(example):
    return f"### Instruction:\n{example['input']}\n\n### Response:\n{example['output']}"

dataset = dataset.map(lambda x: {"text": format_example(x)})

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Load GPT-2 and Tokenizer



In [18]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "gpt2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # GPT-2 doesn't have pad token

model = AutoModelForCausalLM.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Apply LoRA with PEFT

In [ ]:
from peft import get_peft_model, LoraConfig, TaskType

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["c_attn"],  # GPT-2 specific attention module
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

Tokenize and prepare for training

In [28]:
def tokenize(example):
    tokenized = tokenizer(example["text"], truncation=True, padding="max_length", max_length=256)
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized
tokenized_dataset = dataset.map(tokenize, batched=True)

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

 Training with Trainer API

In [29]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./lora-gpt2",
    per_device_train_batch_size=4,
    num_train_epochs=3,
    learning_rate=2e-4,
    logging_steps=10,
    save_steps=50,
    fp16=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer
)

trainer.train()


/tmp/ipython-input-29-3131258210.py:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss


TrainOutput(global_step=3, training_loss=8.947547912597656, metrics={'train_runtime': 28.2667, 'train_samples_per_second': 0.212, 'train_steps_per_second': 0.106, 'total_flos': 786594004992.0, 'train_loss': 8.947547912597656, 'epoch': 3.0})

 Save Adapter

In [30]:
model.save_pretrained("./lora-gpt2-adapter")
tokenizer.save_pretrained("./lora-gpt2-adapter")


('./lora-gpt2-adapter/tokenizer_config.json',
 './lora-gpt2-adapter/special_tokens_map.json',
 './lora-gpt2-adapter/vocab.json',
 './lora-gpt2-adapter/merges.txt',
 './lora-gpt2-adapter/added_tokens.json',
 './lora-gpt2-adapter/tokenizer.json')

In [31]:
from peft import PeftModel
from transformers import pipeline

base_model = AutoModelForCausalLM.from_pretrained("gpt2")
peft_model = PeftModel.from_pretrained(base_model, "./lora-gpt2-adapter")

pipe = pipeline("text-generation", model=peft_model, tokenizer=tokenizer)

prompt = "### Instruction:\nSummarize the legal clause below:\n\n<Insert clause here>\n\n### Response:\n"
output = pipe(prompt, max_new_tokens=100)[0]['generated_text']
print(output)


Device set to use cpu


### Instruction:
Summarize the legal clause below:

<Insert clause here>

### Response:

[Resolved on April 12, 2014 at 12:47 pm.]

The question that arises is whether anyone in the community is aware of the existence of this clause. I am a lawyer.

[Resolved on April 12, 2014 at 12:47 pm.]

The committee is working on that. I want to ask the question:

What is the meaning of a "plaintext" definition?

A "plaintext" definition is one which allows the
